In [1]:
#生成数据集
from mxnet import autograd, nd
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

D:\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 读取数据集
from mxnet.gluon import data as gdata
batch_size = 10                     #设置小批量的大小
dataset = gdata.ArrayDataset(features, labels)    # 将训练数据的特征和标签组合
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)    #加载数据集，并打乱数据集

In [4]:
for X, y in data_iter:
print(X, y)
break


[[ 0.27629048  0.00236695]
 [ 0.7700928   0.9163943 ]
 [ 0.91316134 -1.2948903 ]
 [ 0.2117059  -0.12287327]
 [ 0.30383104 -0.80775887]
 [-0.22538121 -0.15575494]
 [ 1.9141548  -0.8937245 ]
 [-1.1158241   0.2747542 ]
 [-0.6552829  -0.6922099 ]
 [-1.0718824   0.4818612 ]]
<NDArray 10x2 @cpu(0)> 
[ 4.7376227   2.618693   10.442554    5.0448837   7.541996    4.2736063
 11.059692    1.0266986   5.2542806   0.42716137]
<NDArray 10 @cpu(0)>


In [21]:
from mxnet.gluon import nn                 #导入神经网络模块
net = nn.Sequential()                      #实例化Sequential容器
net.add(nn.Dense(1))                       #定义输出层个数为1

In [22]:
#初始化模型参数
from mxnet import init
net.initialize(init.Normal(sigma=0.01))     #随机采样于均值为0、标准差为0.01的正态分布

In [23]:
#定义损失函数
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss() # 平⽅损失⼜称L2范数损失

In [24]:
#定义优化算法
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})   #实例化训练器

In [27]:
#训练模型
num_epochs = 3                        #设置迭代周期为3次
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():       #自动求梯度
            l = loss(net(X), y)       #求单个数据的预测值
        l.backward()                  #反向计算小批量的随机梯度
        trainer.step(batch_size)       #更新参数
    l = loss(net(features), labels)    #损失函数的平均值
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.000049
epoch 2, loss: 0.000048
epoch 3, loss: 0.000049
